\makeatletter
\renewcommand{\@seccntformat}[1]{}
\makeatother

# Program 3: Faktoryzacja $LU$

### Autorzy: Maciej Czyjt, Tomasz Słonina

Dołączenie potrzebnych bibliotek:

``` julia
using LinearAlgebra
```

Plik $matrix\_utils.jl$ zawiera funkcje z poprzednich programów, potrzebne do zrealizowania faktoryzacji: rekurencyjne mnożenie oraz odwracanie macierzy.

``` julia
include("matrix_utils.jl")
```

## $LU$ Factorization

Implementację faktoryzacji LU opisuje pseudokod:

```
// input: A
// output: L, U

function lu_factor_algorithm(A, L, U) {    
    if (A.size > 2) {
        a11, a12, a21, a22 = A[range_A].slice_into_quarters();
        L11, U11 = lu_factor_algorithm(a11);
        L21 = a21 × inverse(U11);
        U12 = inverse(L11) × a12;
        
        S = a22 - (a21 × inverse(U11) × inverse(L11) × a12);
        
        L22, U22 = lu_factor_algorithm(S);
    } else {
        L, U = lu(A);
    }
}
```

Poniżej właściwa implementacja:

``` julia
function lu_factorization(A)
    len = size(A, 1)
    L = zeros(len, len)
    U = zeros(len, len)
    lu_factorization_(A, L, U)
    return L, U
end


function lu_factorization_(A, L, U)
    if size(A, 1) == 2
        l, u = lu(A, NoPivot())
        L[:, :] = l
        U[:, :] = u
        return
    end
    len = (size(A, 1) ÷ 2)
    a11 = [1:len, 1:len]
    a12 = [1:len, len+1:len*2]
    a21 = [len+1:len*2, 1:len]
    a22 = [len+1:len*2, len+1:len*2]

    L11, U11 = lu_factorization(A[a11...])
    inv_U11 = my_inv(U11)
    inv_L11 = my_inv(L11)

    L21 = multiply_rec(A[a21...], inv_U11)
    U12 = multiply_rec(inv_L11, A[a12...])
    S = A[a22...] - multiply_rec(
        A[a21...], 
        multiply_rec(
            inv_U11, 
            multiply_rec(
                inv_L11,
                A[a12...]
            )
        )
    )
    L22, U22 = lu_factorization(S)

    L[a11...] = L11
    L[a21...] = L21
    L[a22...] = L22
    U[a11...] = U11
    U[a12...] = U12
    U[a22...] = U22
end
```

## Testy

Funkcja służąca do testowania implementacji:

``` julia
function test_lu(N)
    for n in 1:N
        A = rand(2^n, 2^n)
        L, U = lu_factorization(A)
        if isapprox(A, L * U)
            println("Test passed for matrix of size ", 2^n, "x", 2^n)
        else
            println("Test failed for matrix of size ", 2^n, "x", 2^n)
        end
    end
end

test_lu(9)
```

Poniżej zostało przedstawione porównanie wyników zaimplementowanej faktoryzaji oraz wersji bibliotecznej. Brane pod uwagę były macierze o rozmiarze $2^k$, dla parametru $k = 1, 2, ..., 9$.

``` julia
function lu_det(A)
    L, U = lu_factorization(A)
    det = 1
    for i in 1:size(A, 1)
        det *= L[i, i] * U[i, i]
    end
    return det
end

lu_dets = []
true_dets = []

N = 9
for n in 1:N
    A = rand(2^n, 2^n)
    push!(lu_dets, lu_det(A))
    push!(true_dets, det(A))
end
```

``` julia
using PlotlyJS

plot(
    table(
        header_values=["N", "LU det", "Julia det", "Error"],
        cells_values=[
            2 .^ (1:N),
            lu_dets,
            true_dets,
            abs.(lu_dets .- true_dets)
        ]
    )
)
```

<!-- ![newplot.png](attachment:newplot.png) -->
 \includegraphics{newplot.png}

### Wykres czasu (w sekundach), potrzebnego do obliczenia macierzy $L$ oraz $U$ w zależności od parametru $k$.

``` julia
function test_time_lu(n)
    times = zeros(n)
    flops = zeros(n)
    for i in 1:n
        A = rand(2^i, 2^i)
        global FLOPS = 0
        times[i] = @elapsed lu_factorization(A)
        flops[i] = FLOPS
        println("n=$i, time=$(times[i]), flops=$(flops[i])")
    end
    return times, flops
end

times, flops = test_time_lu(9)
```

\includegraphics{newplot(2).png}

### Wykres ilości operacji zmiennoprzecinkowych, potrzebnych do obliczenia macierzy $L$ oraz $U$ w zależności od parametru $k$.

\includegraphics{newplot(3).png}

## Wnioski:

Własna implementacja dobrze sprawdza się dla małych rozmiarów macierzy (to jest mniejszych niż $64 \times 64$). Dla większych błąd jest na tyle duży, że może powodować błędy w obliczeniach. Do zastosowań profesjonalnych należy wybierać wersję biblioteczną ze względu na liczne opytmalizacje oraz użycie mechanizmów, dzięki którym algorytm ten jest stabilny numerycznie.